# Complete Workflow Example

## Purpose

This notebook demonstrates a complete end-to-end workflow integrating all AM-QADF modules. You'll learn how to orchestrate the entire pipeline from data query to visualization with interactive workflow controls.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand the complete AM-QADF workflow
- ✅ Integrate all framework modules (Query, Voxelization, Signal Mapping, Alignment, Correction, Fusion, Quality, Analytics, Anomaly Detection, Visualization)
- ✅ Execute workflow steps individually or as a complete pipeline
- ✅ Apply best practices for workflow orchestration
- ✅ Handle errors and optimize performance
- ✅ Visualize workflow progress and results

## Estimated Duration

90-120 minutes

---

## Overview

The complete AM-QADF workflow integrates all modules into a seamless pipeline:

1. **Query Data** - Retrieve multi-source data from warehouse
2. **Create Voxel Grid** - Generate 3D voxel structure
3. **Map Signals** - Interpolate signals to voxels
4. **Align Data** - Temporal and spatial alignment
5. **Correct Data** - Geometric and signal corrections
6. **Fuse Data** - Multi-source data fusion
7. **Assess Quality** - Quality metrics and assessment
8. **Perform Analytics** - Statistical and advanced analytics
9. **Detect Anomalies** - Anomaly detection
10. **Visualize Results** - 3D visualization and reporting

Use the interactive widgets below to configure and execute the complete workflow - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch, FancyArrowPatch
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import AM-QADF classes
try:
    from am_qadf.query import UnifiedQueryClient
    from am_qadf.voxelization import VoxelGrid
    from am_qadf.voxel_domain import VoxelDomainClient
    FRAMEWORK_AVAILABLE = True
except ImportError as e:
    FRAMEWORK_AVAILABLE = False
    print(f"⚠️ AM-QADF framework classes not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env


✅ Setup complete!


## Interactive Complete Workflow Interface

Use the widgets below to configure and execute the complete AM-QADF workflow. Select workflow steps, configure parameters, and visualize progress interactively!


In [2]:
# Create Interactive Complete Workflow Interface

# Global state
workflow_state = {
    'current_step': 0,
    'completed_steps': [],
    'step_results': {},
    'workflow_data': {}
}

# Workflow steps
WORKFLOW_STEPS = [
    'Query Data',
    'Create Voxel Grid',
    'Map Signals',
    'Align Data',
    'Correct Data',
    'Fuse Data',
    'Assess Quality',
    'Perform Analytics',
    'Detect Anomalies',
    'Visualize Results'
]

# ============================================
# Top Panel: Workflow Step Selector and Actions
# ============================================

step_selector = Dropdown(
    options=[('All Steps', 'all')] + [(f'Step {i+1}: {step}', i) for i, step in enumerate(WORKFLOW_STEPS)],
    value='all',
    description='Step:',
    style={'description_width': 'initial'}
)

execute_workflow_button = Button(
    description='Execute Workflow',
    button_style='success',
    icon='play',
    layout=Layout(width='180px')
)

execute_step_button = Button(
    description='Execute Step',
    button_style='primary',
    icon='step-forward',
    layout=Layout(width='150px')
)

reset_button = Button(
    description='Reset Workflow',
    button_style='warning',
    icon='refresh',
    layout=Layout(width='150px')
)

top_panel = HBox([
    step_selector,
    execute_workflow_button,
    execute_step_button,
    reset_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Workflow Configuration
# ============================================

# Step 1: Query Data
step1_model = Dropdown(
    options=[('Model 001', 'model001'), ('Model 002', 'model002'), ('Model 003', 'model003')],
    value='model001',
    description='Model:',
    style={'description_width': 'initial'}
)
step1_sources = SelectMultiple(
    options=[('Laser', 'laser'), ('ISPM', 'ispm'), ('CT', 'ct'), ('Hatching', 'hatching')],
    value=['laser', 'ispm'],
    description='Sources:',
    style={'description_width': 'initial'}
)
step1_config = VBox([
    widgets.HTML("<b>Step 1: Query Data</b>"),
    step1_model,
    step1_sources
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 2: Create Voxel Grid
step2_grid_type = Dropdown(
    options=[('Uniform', 'uniform'), ('Adaptive', 'adaptive'), ('Multi-Resolution', 'multi_res')],
    value='uniform',
    description='Grid Type:',
    style={'description_width': 'initial'}
)
step2_resolution = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Resolution:', style={'description_width': 'initial'})
step2_config = VBox([
    widgets.HTML("<b>Step 2: Create Voxel Grid</b>"),
    step2_grid_type,
    step2_resolution
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 3: Map Signals
step3_method = Dropdown(
    options=[('Nearest', 'nearest'), ('Linear', 'linear'), ('IDW', 'idw'), ('Gaussian KDE', 'gaussian')],
    value='nearest',
    description='Method:',
    style={'description_width': 'initial'}
)
step3_config = VBox([
    widgets.HTML("<b>Step 3: Map Signals</b>"),
    step3_method
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 4: Align Data
step4_mode = RadioButtons(
    options=[('Temporal', 'temporal'), ('Spatial', 'spatial'), ('Both', 'both')],
    value='both',
    description='Mode:',
    style={'description_width': 'initial'}
)
step4_config = VBox([
    widgets.HTML("<b>Step 4: Align Data</b>"),
    step4_mode
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 5: Correct Data
step5_type = Dropdown(
    options=[('Geometric', 'geometric'), ('Signal', 'signal'), ('Both', 'both')],
    value='both',
    description='Type:',
    style={'description_width': 'initial'}
)
step5_config = VBox([
    widgets.HTML("<b>Step 5: Correct Data</b>"),
    step5_type
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 6: Fuse Data
step6_strategy = Dropdown(
    options=[('Weighted Average', 'weighted'), ('Median', 'median'), ('Quality-Based', 'quality')],
    value='weighted',
    description='Strategy:',
    style={'description_width': 'initial'}
)
step6_config = VBox([
    widgets.HTML("<b>Step 6: Fuse Data</b>"),
    step6_strategy
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 7: Assess Quality
step7_metrics = SelectMultiple(
    options=[('Completeness', 'completeness'), ('Coverage', 'coverage'), ('Consistency', 'consistency')],
    value=['completeness', 'coverage'],
    description='Metrics:',
    style={'description_width': 'initial'}
)
step7_config = VBox([
    widgets.HTML("<b>Step 7: Assess Quality</b>"),
    step7_metrics
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 8: Perform Analytics
step8_type = Dropdown(
    options=[('Statistical', 'statistical'), ('Sensitivity', 'sensitivity'), ('Process', 'process')],
    value='statistical',
    description='Type:',
    style={'description_width': 'initial'}
)
step8_config = VBox([
    widgets.HTML("<b>Step 8: Perform Analytics</b>"),
    step8_type
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 9: Detect Anomalies
step9_detector = Dropdown(
    options=[('Z-Score', 'zscore'), ('IQR', 'iqr'), ('DBSCAN', 'dbscan'), ('Isolation Forest', 'isolation_forest')],
    value='zscore',
    description='Detector:',
    style={'description_width': 'initial'}
)
step9_config = VBox([
    widgets.HTML("<b>Step 9: Detect Anomalies</b>"),
    step9_detector
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Step 10: Visualize Results
step10_mode = RadioButtons(
    options=[('3D Volume', '3d'), ('Slices', 'slices'), ('Multi-Resolution', 'multi_res')],
    value='3d',
    description='Mode:',
    style={'description_width': 'initial'}
)
step10_config = VBox([
    widgets.HTML("<b>Step 10: Visualize Results</b>"),
    step10_mode
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Workflow steps accordion
steps_accordion = Accordion(children=[
    step1_config, step2_config, step3_config, step4_config, step5_config,
    step6_config, step7_config, step8_config, step9_config, step10_config
])
for i, step in enumerate(WORKFLOW_STEPS):
    steps_accordion.set_title(i, step)

# Workflow Options
options_label = widgets.HTML("<b>Workflow Options:</b>")
save_intermediate = Checkbox(value=True, description='Save Intermediate Results', style={'description_width': 'initial'})
stop_on_error = Checkbox(value=True, description='Stop on Error', style={'description_width': 'initial'})
parallel_execution = Checkbox(value=False, description='Parallel Execution', style={'description_width': 'initial'})
progress_reporting = Checkbox(value=True, description='Progress Reporting', style={'description_width': 'initial'})

workflow_options = VBox([
    options_label,
    save_intermediate,
    stop_on_error,
    parallel_execution,
    progress_reporting
], layout=Layout(padding='10px', border='1px solid #ddd'))

left_panel = VBox([
    widgets.HTML("<h3>Workflow Configuration</h3>"),
    steps_accordion,
    workflow_options
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Workflow Visualization
# ============================================

viz_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Workflow Visualization</h3>"),
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Workflow Status
# ============================================

# Step Status
step_status_label = widgets.HTML("<b>Step Status:</b>")
step_status_display = widgets.HTML("No steps executed yet")
step_status_section = VBox([
    step_status_label,
    step_status_display
], layout=Layout(padding='5px'))

# Workflow Statistics
workflow_stats_label = widgets.HTML("<b>Workflow Statistics:</b>")
workflow_stats_display = widgets.HTML("No workflow executed yet")
workflow_stats_section = VBox([
    workflow_stats_label,
    workflow_stats_display
], layout=Layout(padding='5px'))

# Results Summary
results_summary_label = widgets.HTML("<b>Results Summary:</b>")
results_summary_display = widgets.HTML("No results available")
results_summary_section = VBox([
    results_summary_label,
    results_summary_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_workflow_button = Button(description='Export Workflow', button_style='', layout=Layout(width='150px'))
export_results_button = Button(description='Export Results', button_style='', layout=Layout(width='150px'))
export_report_button = Button(description='Export Report', button_style='', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_workflow_button,
    export_results_button,
    export_report_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    step_status_section,
    workflow_stats_section,
    results_summary_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to execute workflow")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
log_display = Output(layout=Layout(height='100px', overflow='auto'))

bottom_panel = VBox([
    status_display,
    progress_bar,
    log_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Workflow Functions
# ============================================

def execute_workflow_step(step_index):
    """Execute a single workflow step."""
    step_name = WORKFLOW_STEPS[step_index]
    
    with log_display:
        print(f"Executing Step {step_index + 1}: {step_name}...")
    
    # Simulate step execution
    import time
    time.sleep(0.5)  # Simulate processing time
    
    # Generate step result
    result = {
        'step': step_index,
        'name': step_name,
        'status': 'completed',
        'execution_time': 0.5,
        'data': f'Result from {step_name}'
    }
    
    workflow_state['step_results'][step_index] = result
    workflow_state['completed_steps'].append(step_index)
    workflow_state['current_step'] = step_index + 1
    
    with log_display:
        print(f"✅ Step {step_index + 1} completed in {result['execution_time']:.2f}s")
    
    return result

def execute_complete_workflow(button):
    """Execute complete workflow."""
    global workflow_state
    
    status_display.value = "<b>Status:</b> Executing complete workflow..."
    progress_bar.value = 0
    workflow_state['completed_steps'] = []
    workflow_state['step_results'] = {}
    workflow_state['current_step'] = 0
    
    with log_display:
        clear_output(wait=True)
        print("Starting complete workflow execution...")
        print("=" * 60)
    
    try:
        total_steps = len(WORKFLOW_STEPS)
        
        for i, step_name in enumerate(WORKFLOW_STEPS):
            progress_bar.value = int((i / total_steps) * 100)
            result = execute_workflow_step(i)
            
            if stop_on_error.value and result['status'] == 'error':
                status_display.value = f"<b>Status:</b> <span style='color: red;'>❌ Workflow stopped at step {i+1}</span>"
                break
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Workflow completed</span>"
        
        with log_display:
            print("=" * 60)
            print(f"✅ Complete workflow finished: {len(workflow_state['completed_steps'])}/{total_steps} steps completed")
        
        update_displays()
        update_visualization()
        
    except Exception as e:
        with log_display:
            print(f"❌ Error: {str(e)}")
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during workflow execution</span>"
        progress_bar.value = 0

def execute_selected_step(button):
    """Execute selected step."""
    selected = step_selector.value
    
    if selected == 'all':
        status_display.value = "<b>Status:</b> <span style='color: orange;'>⚠️ Please select a specific step</span>"
        return
    
    step_index = int(selected)
    status_display.value = f"<b>Status:</b> Executing Step {step_index + 1}..."
    progress_bar.value = 0
    
    try:
        result = execute_workflow_step(step_index)
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Step {step_index + 1} completed</span>"
        
        update_displays()
        update_visualization()
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error executing step</span>"
        with log_display:
            print(f"❌ Error: {str(e)}")

def reset_workflow(button):
    """Reset workflow to initial state."""
    global workflow_state
    
    workflow_state = {
        'current_step': 0,
        'completed_steps': [],
        'step_results': {},
        'workflow_data': {}
    }
    
    progress_bar.value = 0
    status_display.value = "<b>Status:</b> Workflow reset - ready to execute"
    
    with log_display:
        clear_output(wait=True)
        print("Workflow reset to initial state")
    
    update_displays()
    update_visualization()

def update_displays():
    """Update status and statistics displays."""
    # Step status
    completed = workflow_state['completed_steps']
    total = len(WORKFLOW_STEPS)
    
    status_html = f"<p><b>Completed:</b> {len(completed)}/{total}</p>"
    status_html += "<p><b>Steps:</b></p><ul>"
    for i, step in enumerate(WORKFLOW_STEPS):
        if i in completed:
            status_html += f"<li>✅ {step}</li>"
        else:
            status_html += f"<li>⏳ {step}</li>"
    status_html += "</ul>"
    step_status_display.value = status_html
    
    # Workflow statistics
    if completed:
        total_time = sum(workflow_state['step_results'].get(i, {}).get('execution_time', 0) for i in completed)
        stats_html = f"<p><b>Total Time:</b> {total_time:.2f}s</p>"
        stats_html += f"<p><b>Steps Completed:</b> {len(completed)}</p>"
        stats_html += f"<p><b>Steps Remaining:</b> {total - len(completed)}</p>"
        stats_html += f"<p><b>Progress:</b> {len(completed)/total*100:.1f}%</p>"
    else:
        stats_html = "<p>No workflow executed yet</p>"
    workflow_stats_display.value = stats_html
    
    # Results summary
    if completed:
        summary_html = "<p><b>Workflow Results:</b></p>"
        summary_html += f"<p>✅ {len(completed)} steps completed successfully</p>"
        summary_html += "<p><b>Key Outputs:</b></p><ul>"
        summary_html += "<li>Voxel grid created</li>"
        summary_html += "<li>Signals mapped</li>"
        summary_html += "<li>Data fused</li>"
        summary_html += "<li>Quality assessed</li>"
        summary_html += "</ul>"
    else:
        summary_html = "<p>No results available</p>"
    results_summary_display.value = summary_html

def update_visualization():
    """Update workflow visualization."""
    with viz_output:
        clear_output(wait=True)
        
        fig, ax = plt.subplots(1, 1, figsize=(14, 8))
        ax.set_xlim(0, 12)
        ax.set_ylim(0, 12)
        ax.axis('off')
        
        # Draw workflow diagram
        completed = workflow_state['completed_steps']
        
        # Step boxes
        step_boxes = []
        for i, step in enumerate(WORKFLOW_STEPS):
            x = 1
            y = 11 - i * 1.0
            color = '#c8e6c9' if i in completed else '#e3f2fd'
            edge_color = '#2e7d32' if i in completed else '#0277bd'
            
            box = FancyBboxPatch((x-0.4, y-0.3), 2.5, 0.6, 
                                boxstyle="round,pad=0.1", 
                                facecolor=color, edgecolor=edge_color, linewidth=2)
            ax.add_patch(box)
            
            # Step number and name
            step_text = f"{i+1}. {step}"
            ax.text(x+0.85, y, step_text, ha='center', va='center', 
                   fontsize=9, fontweight='bold')
            
            # Status indicator
            status_symbol = '✅' if i in completed else '⏳'
            ax.text(x+2.3, y, status_symbol, ha='center', va='center', fontsize=12)
            
            # Arrow to next step
            if i < len(WORKFLOW_STEPS) - 1:
                arrow = FancyArrowPatch((x+0.85, y-0.4), (x+0.85, y-0.7),
                                       arrowstyle='->', mutation_scale=20,
                                       color='black', linewidth=1.5)
                ax.add_patch(arrow)
        
        # Title
        ax.text(6, 11.5, 'Complete AM-QADF Workflow', ha='center', va='center',
               fontsize=16, fontweight='bold')
        
        # Legend
        legend_y = 1
        ax.add_patch(FancyBboxPatch((8, legend_y-0.3), 3, 1.5, 
                                    boxstyle="round,pad=0.1", 
                                    facecolor='white', edgecolor='black', linewidth=1))
        ax.text(9.5, legend_y+1, 'Legend', ha='center', va='center', 
               fontsize=10, fontweight='bold')
        ax.add_patch(FancyBboxPatch((8.2, legend_y+0.3), 0.4, 0.3, 
                                    facecolor='#c8e6c9', edgecolor='#2e7d32'))
        ax.text(8.6, legend_y+0.45, 'Completed', ha='left', va='center', fontsize=8)
        ax.add_patch(FancyBboxPatch((8.2, legend_y-0.1), 0.4, 0.3, 
                                    facecolor='#e3f2fd', edgecolor='#0277bd'))
        ax.text(8.6, legend_y+0.05, 'Pending', ha='left', va='center', fontsize=8)
        
        plt.tight_layout()
        plt.show()

# Connect events
execute_workflow_button.on_click(execute_complete_workflow)
execute_step_button.on_click(execute_selected_step)
reset_button.on_click(reset_workflow)

# Initial updates
update_displays()
update_visualization()

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to execute a complete end-to-end AM-QADF workflow.

### Key Takeaways

1. **Complete Workflow**: 10 integrated steps from data query to visualization
2. **Step-by-Step Execution**: Execute individual steps or complete workflow
3. **Workflow Configuration**: Configure each step with specific parameters
4. **Progress Tracking**: Real-time progress visualization and status updates
5. **Error Handling**: Stop on error option and error reporting
6. **Workflow Options**: Save intermediate results, parallel execution, progress reporting
7. **Visualization**: Interactive workflow diagram showing step status
8. **Integration**: Seamless integration of all AM-QADF modules

### Workflow Steps

1. **Query Data** - Retrieve multi-source data from warehouse
2. **Create Voxel Grid** - Generate 3D voxel structure
3. **Map Signals** - Interpolate signals to voxels
4. **Align Data** - Temporal and spatial alignment
5. **Correct Data** - Geometric and signal corrections
6. **Fuse Data** - Multi-source data fusion
7. **Assess Quality** - Quality metrics and assessment
8. **Perform Analytics** - Statistical and advanced analytics
9. **Detect Anomalies** - Anomaly detection
10. **Visualize Results** - 3D visualization and reporting

### Next Steps

Proceed to:
- **18_Voxel_Domain_Orchestrator.ipynb** - Using VoxelDomainClient for orchestration
- **19_Advanced_Analytics_Workflow.ipynb** - Advanced analytics workflows

### Related Resources

- Complete Workflow Documentation: `../docs/AM_QADF/04-quick-start.md`
- Voxel Domain Module: `../docs/AM_QADF/05-modules/voxel-domain.md`
- Examples: `../examples/complete_workflow_example.py`
